In [11]:
# Import Dependancies
from citipy import citipy 
import os
from config import weather_api_key
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import requests
import time

In [12]:
# Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [13]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [14]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [15]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

762

In [12]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

## Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"] 
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")    

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | hermanus
Processing Record 2 of Set 1 | busselton
Processing Record 3 of Set 1 | svay rieng
Processing Record 4 of Set 1 | ribeira grande
Processing Record 5 of Set 1 | hobart
Processing Record 6 of Set 1 | nemuro
Processing Record 7 of Set 1 | tarudant
City not found. Skipping...
Processing Record 8 of Set 1 | castro
Processing Record 9 of Set 1 | barrow
Processing Record 10 of Set 1 | tsihombe
City not found. Skipping...
Processing Record 11 of Set 1 | mbumi
Processing Record 12 of Set 1 | carnarvon
Processing Record 13 of Set 1 | arraial do cabo
Processing Record 14 of Set 1 | sergeyevka
Processing Record 15 of Set 1 | constitucion
Processing Record 16 of Set 1 | nanakuli
Processing Record 17 of Set 1 | hilo
Processing Record 18 of Set 1 | ushuaia
Processing Record 19 of Set 1 | norman wells
Processing Record 20 of Set 1 | vaini
Processing Record 21 of Set 1 | aldan
Processing Record 22 of Set

In [13]:
len(city_data)

678

In [14]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Hermanus,ZA,-34.4187,19.2345,53.80,83,81,13.98,light rain
1,Busselton,AU,-33.6500,115.3333,60.17,81,100,23.64,light rain
2,Svay Rieng,KH,11.0879,105.7993,81.12,75,100,4.43,overcast clouds
3,Ribeira Grande,PT,38.5167,-28.7000,73.26,90,71,6.40,broken clouds
4,Hobart,AU,-42.8794,147.3294,57.25,79,75,3.00,broken clouds
5,Nemuro,JP,43.3236,145.5750,58.37,98,100,18.45,light rain
6,Castro,BR,-24.7911,-50.0119,57.90,71,86,2.04,overcast clouds
7,Barrow,US,71.2906,-156.7887,41.02,87,100,11.50,mist
8,Mbumi,TZ,-7.9058,39.2105,66.58,95,4,2.37,clear sky
9,Carnarvon,AU,-24.8667,113.6333,64.47,58,99,6.08,overcast clouds


In [19]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")